To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth#installation-instructions---conda).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

**[NEW] Llama-3.1 8b, 70b & 405b are trained on a crazy 15 trillion tokens with 128K long context lengths!**

## Kaggle is slow - you'll have to wait **5 minutes** for it to install.

I suggest you to use our free Colab notebooks instead. I linked our Llama 3.1 8b Colab notebook here: [notebook](https://colab.research.google.com/drive/1Ys44kVvmeZtnICzWz0xgpRnrIOjZAuxp?usp=sharing)

In [1]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

In [2]:
!pip install tokenizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.8/112.8 kB 4.3 MB/s eta 0:00:00


In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.11.11: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.11.11 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset, Dataset
from transformers import TrainingArguments
from trl import SFTTrainer
import pandas as pd
import xformers.ops.fmha as xformers


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `ChatML` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [6]:
from transformers import AutoTokenizer

# Load the correct tokenizer for your model
tokenizer = AutoTokenizer.from_pretrained("unsloth/Meta-Llama-3.1-8B")

# Set EOS token based on the tokenizer
EOS_TOKEN = tokenizer.eos_token if tokenizer.eos_token else "<|endoftext|>"

alpaca_prompt = """You are a cybersecurity expert specializing in cyber threat intelligence. You are given a
multiple-choice question (MCQ) from a Cyber Threat Intelligence (CTI) knowledge benchmark
dataset. Your task is to choose the best option among the four provided. Return your answer as a
single uppercase letter: A, B, C, or D.
### Question:
{Question}
### Input:
A) {Option_A}
B) {Option_B}
C) {Option_C}
D) {Option_D}

### Response :
{GT}
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    questions = examples["Question"]
    option_a = examples["Option A"]
    option_b = examples["Option B"]
    option_c = examples["Option C"]
    option_d = examples["Option D"]
    gts = examples["GT"]  # This is the correct answer
    texts = []
    for question, a, b, c, d, gt in zip(questions, option_a, option_b, option_c, option_d, gts):
        text = alpaca_prompt.format(
            Question=question,
            Option_A=a,
            Option_B=b,
            Option_C=c,
            Option_D=d,
            GT=gt
        ) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

def load_tsv_to_dataset(tsv_path, encoding='ISO-8859-1'):
    df = pd.read_csv(tsv_path, sep='\t', encoding=encoding)
    return Dataset.from_pandas(df)

# Load datasets using 'ISO-8859-1' encoding
train_dataset = load_tsv_to_dataset("/kaggle/input/mineqcm/cti_mcq_final.tsv")
test_dataset = load_tsv_to_dataset("/kaggle/input/qcm-data/cti-mcq.tsv")

# Formatting prompts for training
train_dataset = train_dataset.map(formatting_prompts_func,batched=True)




tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

Map:   0%|          | 0/4776 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        
        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps = 5,
        max_steps = 60,

        learning_rate = 2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/4776 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 4,776 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,2.425900
2,2.480100
3,2.465500
4,2.482300
5,2.408000
6,2.422700
7,2.468100
8,2.476000
9,2.561200
10,2.489700


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [9]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [11]:
model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!


# Prepare for inference
inputs = tokenizer(
    [
        alpaca_prompt.format(
            Question="An adversary leveraging the technique Abuse Elevation Control Mechanism: Setuid and Setgid is targeting which systems from the MITRE ATT&CK Enterprise matrix?", 
            Option_A="Windows",
            Option_B="macOS",
            Option_C="Linux and macOS",
            Option_D="Windows and Linux",
            GT=""  # Leave this blank for generation!
        )
    ],
    return_tensors="pt"
).to("cuda")


from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs ,streamer = text_streamer, max_new_tokens = 128)

==((====))==  Unsloth 2024.11.11: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


/opt/conda/lib/python3.10/site-packages/accelerate/utils/modeling.py:1462: UserWarning: Current model requires 113247040.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

eval


In [12]:
from transformers import TextStreamer
FastLanguageModel.for_inference(model)

# Load the test dataset
test_dataset = load_tsv_to_dataset("/kaggle/input/mydsqcm/qcm_output (1).tsv")

# Prepare to store model's responses
model_responses = []

# Process each example in the test dataset
for example in test_dataset:
    # Prepare the prompt using the Alpaca-style formatting
    prompt = alpaca_prompt.format(
        Question=example["Question"],
        Option_A=example["Option A"],
        Option_B=example["Option B"],
        Option_C=example["Option C"],
        Option_D=example["Option D"],
        GT=""  # Leave this blank for generation
    )
    
    # Tokenize the input prompt
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    
    # Prepare the text streamer for real-time output
    text_streamer = TextStreamer(tokenizer)
    
    # Generate the model's response
    generated_output = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)
    
    # Decode the generated response
    response = tokenizer.decode(generated_output[0], skip_special_tokens=True)
    
    # Store the response
    model_responses.append(response)

# Save the results to a CSV or TSV file
output_df = pd.DataFrame({
    "Question": test_dataset["Question"],
    "Model_Response": model_responses
})

# Save the responses to a file
output_df.to_csv("model_responses.csv",index=False)

<|begin_of_text|>You are a cybersecurity expert specializing in cyber threat intelligence. You are given a
multiple-choice question (MCQ) from a Cyber Threat Intelligence (CTI) knowledge benchmark
dataset. Your task is to choose the best option among the four provided. Return your answer as a
single uppercase letter: A, B, C, or D.
### Question:
1**
### Input:
A) Impersonating a user
B) Compromising a host
C) Manipulating data
D) Gaining persistence

### Response :

### Solution:
A

### Explanation:
Impersonating a user is a common technique used by attackers to gain unauthorized access to
systems or networks. It involves assuming the identity of a legitimate user, usually by obtaining
their credentials through social engineering or other means. Once the attacker has gained access
to the system, they can perform actions or access data as if they were the legitimate user, thereby
circumventing security measures and bypassing authorization controls.

### Solution:
B

### Explanation:
Com

In [15]:
import re
import pandas as pd

# Load the model responses TSV file
model_responses_df = pd.read_csv("/kaggle/working/model_responses.csv", sep='\t')

# Prepare to store cleaned responses
cleaned_responses = []

# Iterate over each row in the dataframe
for response in model_responses_df["Model_Response"]:
    # Use regular expression to extract the response after '### Response:'
    match = re.search(r'### Response:\s*([A-D])', response)
    
    if match:
        # Append the extracted response (A, B, C, or D) to the cleaned_responses list
        cleaned_responses.append(match.group(1))
    else:
        # If no match is found, append an empty string or placeholder
        cleaned_responses.append("")

# Create a new DataFrame with the cleaned responses
cleaned_df = pd.DataFrame({
    "Model_Response": cleaned_responses
})

# Save the cleaned responses to a new TSV file
cleaned_df.to_csv("cleaned_model_responses.tsv", sep='\t', index=False)

print("Cleaned responses saved to cleaned_model_responses.tsv")

KeyError: 'Model_Response'